In [9]:
import os
import datasets
import logging
import torch
import tensorflow as tf
import pandas as pd
import random
import transformers
import jsonlines

from utilities import tokenize_and_split_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

logger = logging.getLogger(__name__)

In [4]:
import torch

def check_gpu():
    if torch.cuda.is_available():
        print(f"CUDA is available. PyTorch version: {torch.__version__}")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"  Memory Allocated: {torch.cuda.memory_allocated(i) / 1024 ** 2:.2f} MB")
            print(f"  Memory Cached: {torch.cuda.memory_reserved(i) / 1024 ** 2:.2f} MB")
    else:
        print("CUDA is not available. Using CPU.")

check_gpu()


CUDA is available. PyTorch version: 2.3.0+cu118
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3070 Ti Laptop GPU
  Memory Allocated: 0.00 MB
  Memory Cached: 0.00 MB


## Load Dataset and Model

In [10]:
dataset_path = 'kotzeje/lamini_docs.jsonl'
use_hf = True

model_name = 'EleutherAI/pythia-70m'
base_model = AutoModelForCausalLM.from_pretrained(model_name)

## Set up the training config and tokenizer

In [11]:
training_config = {
    'model' : {
        'pretrained_name' : model_name,
        'max_length' : 2048
    },
    'datasets' : {
        'use_hf' : use_hf,
        'path' : dataset_path
    },
    'verbose' : True
}

In [44]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_dataset(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 900
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


## Define function to carry out inference

In [35]:
def inference(text, model, tokenizer, max_input_tokens = 1000, max_output_tokens = 100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move the model to the appropriate device
    model.to(device)
    input_ids = tokenizer.encode(
        text,
        return_tensors='pt',
        truncation=True,
        max_length = max_input_tokens
    ).to(device)

    generated_tokens_with_prompt = model.generate(
        input_ids=input_ids,
        max_length = max_output_tokens
    ).to(device)

    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer

## Try base model

In [14]:
test_text = test_dataset[1]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Are there any alternatives to Lamini that offer similar functionality?
Correct answer from Lamini docs: Lamini provides support for any tasks that can be completed by an LLM. If you think a recommendation system can be built using a LLM, Lamini can help you train the model on your data. If user context or behavior can be contextualized into text data, we think this is possible.
Model's answer: 


A:

I think you should use the following code:
public class Lamini
{
    public string Name { get; set; }
    public string Description { get; set; }
    public string Description { get; set; }
    public string Description { get; set; }
    public string Description { get; set; }
    public string Description { get; set


## Setup training

In [26]:
max_steps = 100
trained_model_name = f'lamini_docs_{max_steps}_steps'
output_dir = 'output/saved_models/' + trained_model_name

In [27]:
training_args = TrainingArguments(

    learning_rate=1.0e-5,
    num_train_epochs=1,
    max_steps=max_steps,
    per_device_train_batch_size=1,
    output_dir=output_dir,
    
    overwrite_output_dir=False,
    disable_tqdm=False,
    eval_steps=120,
    save_steps=120,
    warmup_steps=1,
    per_device_eval_batch_size=1,
    evaluation_strategy='steps',
    logging_strategy='steps',
    logging_steps=1,
    optim='adafactor',
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,

    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='eval_loss',
    greater_is_better=False
    

)

In [18]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [28]:
trainer = Trainer(
    model=base_model,
    # model_flops=model_flops,
    # total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

max_steps is given, it will override any value given in num_train_epochs


## Training

In [29]:
training_output = trainer.train()

                                     
  0%|          | 0/3 [01:25<?, ?it/s]          

{'loss': 5.6333, 'grad_norm': 447.4446105957031, 'learning_rate': 1e-05, 'epoch': 0.0}


                                     
  0%|          | 0/3 [01:26<?, ?it/s]          

{'loss': 5.5331, 'grad_norm': 418.32513427734375, 'learning_rate': 9.8989898989899e-06, 'epoch': 0.01}


                                     
  0%|          | 0/3 [01:28<?, ?it/s]          

...

{'loss': 0.6967, 'grad_norm': 9.130102157592773, 'learning_rate': 2.0202020202020205e-07, 'epoch': 0.44}


                                     
  0%|          | 0/3 [05:01<?, ?it/s]           

{'loss': 0.2863, 'grad_norm': 5.3537726402282715, 'learning_rate': 1.0101010101010103e-07, 'epoch': 0.44}


                                     
                                                 
100%|██████████| 100/100 [03:39<00:00,  2.20s/it]

{'loss': 0.5241, 'grad_norm': 7.851963043212891, 'learning_rate': 0.0, 'epoch': 0.44}
{'train_runtime': 219.8134, 'train_samples_per_second': 1.82, 'train_steps_per_second': 0.455, 'train_loss': 0.6466012197732925, 'epoch': 0.44}


## Testing after trianing for 100 steps

In [37]:
test_text = test_dataset[1]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Are there any alternatives to Lamini that offer similar functionality?
Correct answer from Lamini docs: Lamini provides support for any tasks that can be completed by an LLM. If you think a recommendation system can be built using a LLM, Lamini can help you train the model on your data. If user context or behavior can be contextualized into text data, we think this is possible.
Model's answer: 
Yes, there are several options available for Lamini to offer similar functionality.


In [43]:
test_text = test_dataset[4]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[4]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Are there any examples of using Lamini for language translation tasks?
Correct answer from Lamini docs: Language translation is a great use case for a language model. Once you’ve exhausted the benefits of prompt tuning, you may use Lamini to fine-tune a fully multilingual language model.
Model's answer: 
Yes, Lamini is a language model for language translation tasks. Lamini is a language model for language translation tasks. It is designed to be used in a specific language model. It is designed to be used in a specific language model. It is designed to be used in a specific language model. It is designed to be used in a specific language model. It is designed to be used in a specific language model.
